In [ ]:
#|default_exp migrate

# migrate
> Utilities for migrating to nbdev

In [ ]:
#|export
from nbdev.process import first_code_ln
from nbdev.processors import nb_fmdict, construct_fm, insert_frontmatter, is_frontmatter, yml2dict, filter_fm
from nbdev.config import get_config, read_nb
from nbdev.sync import write_nb
from nbdev.clean import process_write
from nbdev.showdoc import show_doc
from fastcore.all import *
import shutil

## Convert fastpages blog posts to nbdev

In [ ]:
#|export
def _get_fm(path): return nb_fmdict(read_nb(path), remove=False)
def _get_raw_fm(nb): 
    return first(L(nb.cells).filter(lambda x: x.cell_type == 'raw')).source

In [ ]:
#|export
def _cat_slug(fmdict):
    "Get the partial slug from the category front matter."
    return '/' + '/'.join(sorted(fmdict.get('categories', '')))

In [ ]:
#|hide
_fm1 = _get_fm('../../tests/2020-09-01-fastcore.ipynb')
test_eq(_cat_slug(_fm1), '/fastai/fastcore')

_fm2 = _get_fm('../../tests/2020-02-20-test.ipynb')
test_eq(_cat_slug(_fm2), '/jupyter')

In [ ]:
#|export
def _file_slug(fname): 
    "Get the partial slug from the filename."
    p = Path(fname)
    dt = '/'+p.name[:10].replace('-', '/')+'/'
    return dt + p.stem[11:]    

In [ ]:
#|hide
test_eq(_file_slug('../../tests/2020-09-01-fastcore.ipynb'), 
        '/2020/09/01/fastcore')

In [ ]:
#|export
def _add_alias(fm:dict, path:Path):
    if 'permalink' in fm: fm['aliases'] = [f"{fm.pop('permalink').strip()}"]
    else: fm['aliases'] = [f'{_cat_slug(fm) + _file_slug(path)}']

### Migrate notebooks

In [ ]:
#|export
def migrate_nb_fm(path, overwrite=True):
    "Migrate fastpages front matter in notebooks to a raw cell."
    nb = read_nb(path)
    fm = nb_fmdict(nb)
    _add_alias(fm, path)
    insert_frontmatter(nb, fm_dict=fm)
    if overwrite: write_nb(nb, path)
    return nb

In [ ]:
_nb = migrate_nb_fm('../../tests/2020-09-01-fastcore.ipynb', overwrite=False)
print(_get_raw_fm(_nb))

---
aliases:
- /fastcore/
author: <a href='https://twitter.com/HamelHusain'>Hamel Husain</a>
categories:
- fastcore
- fastai
comments: true
description: A unique python library that extends the python programming language
  and provides utilities that enhance productivity.
image: images/copied_from_nb/fastcore_imgs/td.png
title: 'fastcore: An Underrated Python Library'

---


In [ ]:
_nb = migrate_nb_fm('../../tests/2020-02-20-test.ipynb', overwrite=False)
print(_get_raw_fm(_nb))

---
aliases:
- /jupyter/2020/02/20/test
categories:
- jupyter
comments: true
description: A tutorial of fastpages for Jupyter notebooks.
image: images/chart-preview.png
title: Fastpages Notebook Blog Post

---


### Migrate Markdown Files

In [ ]:
#|export
_re_fm_md = re.compile(r'^---(.*\S+.)?---', flags=re.DOTALL)

def _md_fmdict(txt):
    "Get front matter as a dict from a markdown file."
    m = _re_fm_md.match(txt)
    if m: return yml2dict(m.group(1))
    else: return {}

In [ ]:
#|hide
_mdtxt = Path('../../tests/2020-01-14-test-markdown-post.md').read_text()
_res = _md_fmdict(_mdtxt)

In [ ]:
#|hide
test_eq(_res, {'toc': True,
               'layout': 'post',
               'description': 'A minimal example of using markdown with fastpages.',
               'categories': ['markdown'],
               'title': 'An Example Markdown Post'})

In [ ]:
#|export
def migrate_md_fm(path, overwrite=True):
    "Make fastpages front matter in markdown files quarto compliant."
    p = Path(path)
    md = p.read_text()
    fm = _md_fmdict(md)
    if fm:
        _add_alias(fm, path)
        txt = _re_fm_md.sub(construct_fm(filter_fm(fm)), md)
        if overwrite: p.write_text(txt)
        return txt
    else: return md 

Here is what the front matter of a markdown post looks like before:

In [ ]:
#|eval: false
print(run('head -n13 ../../tests/2020-01-14-test-markdown-post.md'))

---
toc: true
layout: post
description: A minimal example of using markdown with fastpages.
categories: [markdown]
title: An Example Markdown Post
---
# Example Markdown Post

## Basic setup

Jekyll requires blog post files to be named according to the following format:


And this is what it looks like after:

In [ ]:
_res = migrate_md_fm('../../tests/2020-01-14-test-markdown-post.md', overwrite=False)
print(_res[:300])

---
aliases:
- /markdown/2020/01/14/test-markdown-post
categories:
- markdown
description: A minimal example of using markdown with fastpages.
title: An Example Markdown Post

---
# Example Markdown Post

## Basic setup

Jekyll requires blog post files to be named according to the following format:



In [ ]:
#|hide
assert """---
aliases:
- /markdown/2020/01/14/test-markdown-post
categories:
- markdown
description: A minimal example of using markdown with fastpages.
title: An Example Markdown Post

---""" in _res

## Convert nbdev v1 projects to nbdev v2

### Directives

nbdev v2 directives start with a `#|` whereas v1 directives were comments without a pipe `|`.

In [ ]:
#|export
_alias = merge({k:'code-fold: true' for k in ['collapse', 'collapse_input', 'collapse_hide']}, {'collapse_show':'code-fold: show'})
def _subv1(s): return _alias.get(s, s)

In [ ]:
#|export
def _re_v1():
    d = ['default_exp', 'export', 'exports', 'exporti', 'hide', 'hide_input', 'collapse_show', 'collapse',
         'collapse_hide', 'collapse_input', 'hide_output',  'default_cls_lvl']
    d += L(get_config().tst_flags).filter()
    d += [s.replace('_', '-') for s in d] # allow for hyphenated version of old directives
    _tmp = '|'.join(list(set(d)))
    return re.compile(f"^[ \f\v\t]*?(#)\s*({_tmp})(?!\S)", re.MULTILINE)

def _repl_directives(code_str): 
    def _fmt(x): return f"#| {_subv1(x[2].replace('-', '_').strip())}"
    return _re_v1().sub(_fmt, code_str)

In [ ]:
show_doc(_repl_directives)

---

#### _repl_directives

>      _repl_directives (code_str)

for example, if any of the lines below are valid nbdev v1 directives, they replaced with a `#|`:

In [ ]:
#|hide
_test_dir = """
#default_exp
 #export
# collapse-show
#collapse-hide
#collapse
# collapse_output
not_dir='#export'
# hide_input
foo
"""
test_eq(_repl_directives(_test_dir),
"""
#| default_exp
#| export
#| code-fold: show
#| code-fold: true
#| code-fold: true
# collapse_output
not_dir='#export'
#| hide_input
foo
""")

In [ ]:
#|export
def _repl_v1dir(nb):
    "Replace nbdev v1 with v2 directives."
    for cell in nb['cells']:
        if cell.get('source') and cell.get('cell_type') == 'code':
            ss = cell['source'].copy()
            first_code = first_code_ln(ss, re_pattern=_re_v1())
            if not first_code: first_code = len(ss)
            if not ss: pass
            else: cell['source'] = [_repl_directives(c) for c in ss[:first_code]] + ss[first_code:]

In [ ]:
#|hide
_code = _test_dir.splitlines(True)

tst = {'cell_type': 'code', 'execution_count': 26,
       'metadata': {'hide_input': True, 'meta': 23},
       'outputs': [{'execution_count': 2,
                    'data': {
                        'application/vnd.google.colaboratory.intrinsic+json': {'type': 'string'},
                        'plain/text': ['sample output',]
                    }, 'output': 'super'}],
       'source': _code}
nb = {'metadata': {'kernelspec': 'some_spec', 'jekyll': 'some_meta', 'meta': 37}, 'cells': [tst]}

_repl_v1dir(nb)
test_eq(nb['cells'][0], {'cell_type': 'code',
     'execution_count': 26,
     'metadata': {'hide_input': True, 'meta': 23},
     'outputs': [{'execution_count': 2,
     'data': {'application/vnd.google.colaboratory.intrinsic+json': {'type': 'string'},
     'plain/text': ['sample output']},
     'output': 'super'}],
     'source': ['\n',
  '#| default_exp\n',
  '#| export\n',
  '#| code-fold: show\n',
  '#| code-fold: true\n',
  '#| code-fold: true\n',
  '# collapse_output\n',
  "not_dir='#export'\n",
  '# hide_input\n', #after the first line of code there are no directives to migrate
  'foo\n']
    })

## Callouts

In fastpages, there was a markdown shortuct for callouts for `Note`, `Tip`, `Important` and `Warning` with block quotes.  Since Quarto has its own [callout blocks](https://quarto.org/docs/authoring/callouts.html#callout-types) with markdown syntax, we do not implement these shortcuts in nbdev.  Instead, we offer a manual conversion utility for these callouts so that you can migrate from fastpages to Quarto.

In [ ]:
#|export
_re_callout = re.compile(r'^>\s(Warning|Note|Important|Tip):(.*)', flags=re.MULTILINE)
def _co(x): return ":::{.callout-"+x[1].lower()+"}\n\n" + f"{x[2].strip()}\n\n" + ":::"
def _convert_callout(s): 
    "Convert nbdev v1 to v2 callouts."
    return _re_callout.sub(_co, s)

In [ ]:
show_doc(_convert_callout)

---

#### _convert_callout

>      _convert_callout (s)

Convert nbdev v1 to v2 callouts.

For example, the below markdown:

In [ ]:
_callouts="""
## Boxes / Callouts

> Warning: There will be no second warning!

Other text

> Important: Pay attention! It's important.

> Tip: This is my tip.

> Note: Take note of `this.`
"""

Gets converted to:

In [ ]:
#| echo:false
_c = _convert_callout(_callouts)
assert '> Tip:' not in _c
assert 'Other text' in _c
print(_c[:156])


## Boxes / Callouts

:::{.callout-warning}

There will be no second warning!

:::

Other text

:::{.callout-important}

Pay attention! It's important.

:::


In [ ]:
#|export
def _repl_v1callouts(nb):
    "Replace nbdev v1 with v2 callouts."
    for cell in nb['cells']:
        if cell.get('source') and cell.get('cell_type') == 'markdown':
            cell['source'] = [_convert_callout(c) for c in cell['source'].copy()]
    return nb

In [ ]:
#|export
@call_parse
def nbdev_migrate(
    fname:str=None, # A notebook name or glob to migrate
    disp:bool=False,  # Print the outputs with newly formatted directives
    stdin:bool=False, # Read notebook from input stream
    no_skip:bool=False, # Do not skip directories beginning with an underscore
):
    "Convert all directives and callouts in `fname` from v1 to v2"
    _migrate = compose(_repl_v1callouts, _repl_v1dir)
    _write = partial(process_write, warn_msg='Failed to replace directives', proc_nb=_migrate)
    if stdin: _write(f_in=sys.stdin, f_out=sys.stdout)
    _skip_re = None if no_skip else '^[_.]'
    if fname is None: fname = get_config().path('nbs_path')
    for f in globtastic(fname, file_glob='*.ipynb', skip_folder_re=_skip_re): _write(f_in=f, disp=disp)

In [ ]:
#|hide
_orig =  Path('../../tests/2020-02-20-test.ipynb') # nbdev v1 notebook
_tmp =  Path('../../tests/2020-02-20-test-COPY.ipynb') # A copy of this nb that will be migrated
shutil.copy(_orig, _tmp)
nbdev_migrate(_tmp)

def _nb2str(p): return str(L(read_nb(p).cells).attrgot('source'))
assert ':::{.callout-warning}' not in _nb2str(_orig) and ':::{.callout-warning}' in _nb2str(_tmp)
assert '#| code-fold: true' not in _nb2str(_orig) and '#| code-fold: true' in _nb2str(_tmp)
assert '#| single-value' not in _nb2str(_tmp)

if _tmp.exists(): _tmp.unlink() # missing_ok not in python 3.7

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()